In [1]:
from datetime import datetime
import json
import math
import pandas as pd
import requests
from tqdm.notebook import tqdm

In [2]:
url = "https://histdatafeed.vps.com.vn/industry"
response = requests.get(url)
data = response.json()
industries = data.get('data', [])
industries_data_frame = pd.DataFrame(industries)
industries_data_frame.rename(columns={"IcbName": "name", "IcbNameEn": "english_name"})
industries_data_frame.to_csv("./csv/industries.csv", index=False)

In [3]:
url = "https://bgapidatafeed.vps.com.vn/listvn30"
response = requests.get(url)
vn30 = response.json()
vn30_file = open("./vn30.json", "w")
vn30_json = json.dumps(vn30, indent=2)
vn30_file.write(vn30_json)
vn30_file.close()

In [4]:
url = "https://bgapidatafeed.vps.com.vn/getlistallstock"
response = requests.get(url)
companies : list[dict] = response.json()
companies_data_frame = pd.DataFrame(companies)
companies_data_frame.to_csv("./csv/companies.csv", index=False)

In [5]:
timestamp = math.floor(datetime.now().timestamp())
timestamp

1700888350

In [6]:
def get_history(symbol : str):
    YEAR = 2023
    years = range(2000, YEAR + 1)
    now = math.floor(datetime.now().timestamp())
    url = f"https://histdatafeed.vps.com.vn/tradingview/history?symbol={symbol}&resolution=1D&from=0&to={now}"
    response = requests.get(url)
    history_data : dict = response.json()
    symbol = history_data.get("symbol", "")
    t : list[int] = history_data.get("t", [])
    o : list[float] = history_data.get("o", [])
    h : list[float] = history_data.get("h", [])
    l : list[float] = history_data.get("l", [])
    c : list[float] = history_data.get("c", [])
    v : list[int] = history_data.get("v", [])
    history : list[dict] = []
    if len(t) > 0:
        for index, timestamp in enumerate(t):
            date = datetime.fromtimestamp(timestamp).date()
            history.append({
                "date": date,
                "timestamp": timestamp,
                "open": o[index],
                "high": h[index],
                "low": l[index],
                "close": c[index],
                "volume": v[index]
            })
        history_data_frame = pd.DataFrame(history)
        history_data_frame['date'] = pd.to_datetime(history_data_frame['date'])
        history_data_frame.to_csv(f"./csv/history/all/{symbol}.csv", index=False)
        for year in years:
            year_data_frame = history_data_frame[history_data_frame['date'].dt.year == year]
            if year_data_frame.size > 0:
                year_data_frame.to_csv(f"./csv/history/{year}/{symbol}.csv", index=False)

In [7]:
for symbol in tqdm(vn30):
    get_history(symbol)

0
0
0
0
0
0
203
1736
1736
1757
1750
1736
1750
1750
1729
1736
1757
1750
1750
1750
1799
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1533
1750
1715
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1617
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
945
1750
1736
1750
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
840
1750
1736
1750
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
91
1736
1736
1757
1750
1736
1750
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
1113
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1393
1750
1722
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2352
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
357
1736
1757
1750
1736
1750
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
308
1750
1750
1729
1736
1757
1750
1736
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
287
1750
1736
1750
1750
1729
1736
1757
1750
1736
1750

In [8]:
list_of_stock = list(filter(lambda c: len(c.get("stock_code")) == 3, companies))
len(list_of_stock)

1615

In [9]:
list_of_stock.reverse()
for stock in tqdm(list_of_stock):
    symbol : str = stock.get("stock_code", "")
    get_history(symbol)

  0%|          | 0/1615 [00:00<?, ?it/s]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
742
1750
1750
1764
1743
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
931
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
651
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
1120
1757
1750
1736
1750
1750
1589
1736
1757
1750
1750
1750
1764
1743
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
189
1736
1757
1750
1750
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
686
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
35
1757
1750
1750
1750
1764
1743
1743
1568
0
0
0
0
0
0
0
49
1736
1757
1750
1736
1750
1498
0
0
0
0
0
924
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1106
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
294
1750
1750
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
350
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
154
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1554
1750
1750
1764
1743
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
868
1750
1764
1750
1743
1568
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
